# llm 기본틀 만들기
- OpenAI Key 로딩, ChatOpenAI() 객체 생성

In [29]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

load_dotenv('../.env', override=True)

api_key = os.getenv('OPENAI_API_KEY')
# print(api_key)

# 모델 객체 생성

In [30]:
# gpt model 객체 생성
# llm = ChatOpenAI(model="gpt-4o", api_key=api_key)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key=api_key)
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000025C94CEA090>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000025C94CE8F50>, root_client=<openai.OpenAI object at 0x0000025C94CE8410>, root_async_client=<openai.AsyncOpenAI object at 0x0000025C94CEA3F0>, model_name='gpt-3.5-turbo-0125', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [31]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 시스템 프롬프트 작성

- https://github.com/f/awesome-chatgpt-prompts
검색 : financial
```
Want assistance provided by qualified individuals enabled with experience on understanding charts using technical analysis tools while interpreting macroeconomic environment prevailing across world consequently assisting customers acquire long term advantages requires clear verdicts therefore seeking same through informed predictions written down precisely! First statement contains following content- "Can you tell us what future stock market looks like based upon current conditions ?".
```

In [32]:
system_prompt = """
Want assistance provided by qualified individuals enabled with experience on understanding charts using technical analysis tools while interpreting macroeconomic environment prevailing across world consequently assisting customers acquire long term advantages requires clear verdicts therefore seeking same through informed predictions written down precisely! First statement contains following content- “Can you tell us what future stock market looks like based upon current conditions ?".
"""
user_prompt = """
    {company}에 주식을 투자해도 될까요? 마크다운 형식의 투자보고서를 한글로 작성해 주세요.
    야래의 기본 정보, 재무제표를 참고해 마크다운 형식의 투자 보고서를 한글로 작성해 주세요.

    기본정보:
    {basic_info}

    재무제표:
    {finacial_statement}
"""

# 프롬프트 객체 생성

In [33]:
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", user_prompt)
])
prompt

ChatPromptTemplate(input_variables=['basic_info', 'company', 'finacial_statement'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='\nWant assistance provided by qualified individuals enabled with experience on understanding charts using technical analysis tools while interpreting macroeconomic environment prevailing across world consequently assisting customers acquire long term advantages requires clear verdicts therefore seeking same through informed predictions written down precisely! First statement contains following content- “Can you tell us what future stock market looks like based upon current conditions ?".\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['basic_info', 'company', 'finacial_statement'], input_types={}, partial_variables={}, template='\n    {company}에 주식을 투자해도 될까요? 마크다운 형식의 투자보고서를 한글로 작성해 주세요.\n    야래

# OutPut 파서 객체 생성

In [34]:
output_parser = StrOutputParser()
output_parser

StrOutputParser()

# LCEL로 LLM 요청 및 응답받기

- chain 구성 : LCEL(LangChain Expression Language)

In [35]:
# from stock_info import Stock 인식이 안될 경우 아래 코드 실행
import sys
import os

# 상위 디렉터리(LANGCHAIN_PJT)를 경로에 추가
if '__file__' in globals():
    # sys.path.append(os.path.dirname(os.path.abspath(__file__)))
    sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
else:
    # 예: 노트북 환경일 경우 현재 작업 디렉토리 추가
    sys.path.append(os.getcwd())


In [36]:
from stock_info import Stock

# 회사 이름
company = "MicroSoft"
symbol = "MSFT"

stock = Stock(symbol)
stock


In [ ]:

# chain 구성 : LCEL(LangChain Expression Language)객체 생성
chain = prompt | llm | output_parser

# 회사 이름
# company = "MicroSoft"
# symbol = "MSFT"  # stock의 symbol 정보

company = "MicroSoft"
symbol = "MSFT"
# 기본정보 :  basic_info
# 재무제표: finacial_statement

# stock 정보를 객체로 모듈화해서 불러오기
from stock_info import Stock

stock = Stock(symbol)

req_value = {
    "company":company,
    # 기본정보 :  basic_info
    "basic_info": stock.get_basic_info(),
    # 재무제표: finacial_statement
    "finacial_statement" : stock.get_financial_statement()
            }


response = chain.invoke(req_value)


In [38]:
print(response)

# Microsoft Corporation 투자보고서

## 기본정보:
- 기업명: Microsoft Corporation
- 시가총액: 3,512,767,086,592
- 산업: 소프트웨어 - 인프라스트럭쳐
- 섹터: 기술
- 정규직 종업원 수: 228,000명
- 현재 주가: $472.62
- 기업가치: 3,525,534,547,968

## 재무제표:

### 손익계산서
| 항목               | 2024년              | 2023년              | 2022년             | 2021년             |
|-------------------|---------------------|---------------------|--------------------|--------------------|
| 총 수익            | $245,122,000,000    | $211,915,000,000    | $198,270,000,000  | $168,088,000,000  |
| 총 이익            | $171,008,000,000    | $146,052,000,000    | $135,620,000,000  | $115,856,000,000  |
| 영업 이익          | $109,433,000,000    | $88,523,000,000     | $83,383,000,000   | $69,916,000,000   |
| 순 이익            | $88,136,000,000     | $72,361,000,000     | $72,738,000,000   | $61,271,000,000   |

### 대차대조표
| 항목                                       | 2024년              | 2023년              | 2022년             | 2021년             |
|-----------------------